### Template notebook: running a simulation

Use this notebook as a starting point to configure and run a single simulation. The typical workflow is:
1. Set model, network, noise, and integrator parameters.
2. Build the network and system functions.
3. Run the integrator and save outputs.

The output files can be read even if the simulations are still running. Use the other template notebook (template_data_analysis.ipynb) to do so!  

In [ ]:
# Core library imports for dynamics, noise, integrator, and I/O
from dyn_net.dynamical_systems import get_drift
from dyn_net.noise import get_noise
from dyn_net.integrator.params import EulerMaruyamaParams
from dyn_net.dynamical_systems.stats_registry import get_stats
from dyn_net.utils.stats import open_stats_writer, close_stats_writer
from dyn_net.utils.state import open_state_writer, close_state_writer
from dyn_net.networks import get_network
from dyn_net.utils.network import update_system_params_for_network

# Numerical tools
import numpy as np
from pathlib import Path
import json

# JIT-accelerated Kuramoto helpers and integrator
from dyn_net.dynamical_systems.jit_kuramoto import kuramoto_chunk, build_kuramoto_kernel_params
from dyn_net.integrator.jit import integrate_chunked_jit_timed



The cell below allows to modify the parameters of the system, the integrator and I/O options

In [ ]:
# Number of agents (size of the full system)
n = 1000

# Dynamical system definition
system = "kuramoto" # change this to change dynamical system under investigation
params_system = {
    "theta": 0.3, # interaction strength
}

# Noise definition
params_noise = {
    "sigma": 0.1,
}

# Network definition
network_name = "bistable_graphon" # selects the topology of the graph, available ones can be found in src/dyn_net/networks/registry
network_params = { # each topology has its own parameters which can be specified in this dictionary. If not specified defaults to default values
    "n": n,
    "sort_latent" : True
}

"""
The bistable graphon has the following structure

def _w(x: np.ndarray, y: np.ndarray, p: BistableGraphonParams) -> np.ndarray:
    # Symmetric graphon with three peaks.
    return (
        p.amplitude_1
        * np.exp(-((x - p.mean_1) ** 2 + (y - p.mean_1) ** 2) / p.var_1)
        + p.amplitude_2
        * np.exp(-((x - p.mean_2) ** 2 + (y - p.mean_2) ** 2) / p.var_2)
        + p.amplitude_3
        * np.exp(-((x - p.mean_3) ** 4 + (y - p.mean_3) ** 4) / p.var_3)
    )

with default values given by (the ones that were available in )

class BistableGraphonParams(BaseModel):
    model_config = ConfigDict(extra="forbid")
    n: int = Field(..., ge=1)
    rho_n: float = Field(1.0, gt=0.0)
    amplitude_1: float = 0.2
    mean_1: float = 0.2
    var_1: float = 0.02
    amplitude_2: float = 0.1
    mean_2: float = 0.5
    var_2: float = 0.02
    amplitude_3: float = 0.2
    mean_3: float = 0.8
    var_3: float = 0.0005
    sort_latent : bool = False # whether to sort latent variables
    seed: int | None = None # seed for the sampling of the graph

If you want to change any of these values, you can just specify them in network_params above by introducing a new field
"""


# Output directory and file names
output_dir = Path(".")  # change to the folder you want the results to be saved to
output_dir.mkdir(parents=True, exist_ok=True)
stats_name = "stats.h5"
state_name = "state.h5"
stats_path = output_dir / stats_name
state_path = output_dir / state_name

# Integrator parameters
p_int = EulerMaruyamaParams(
    tmin=0.0,
    tmax=10000.0,
    dt=0.1,
    stats_every=10, # this saves the statistics of the Kuramoto model (classic ones) every stats_every steps, in this case 10*0.1 = 1 time units
    state_every=10, # this saves the n particle state of the system every state_every step
    write_stats_at_start=True,
    write_state_at_start=True,
)


# Initial condition for all agents (uniform on [0, 2pi))
x0 = np.random.uniform(0.0, 2 * np.pi, size=n)


The cell below prepares the full system and sets the integrator to the correct dynamical system. Opens the files where the data will be saved as the simulation runs.

In [ ]:
# Build the network adjacency/weight matrix A from the chosen topology
build_net, p_net = get_network(network_name, network_params)
A = build_net(p_net)

# Add the adjacency matrix to the dynamical system (adjust scaling factor for sparse matrices too)
update_system_params_for_network(params_system, network_name, network_params, A)

# Build drift, noise, and statistics functions
F, pF = get_drift(system, params_system)
G, pG = get_noise("additive_gaussian", params_noise)
stats_fn, stats_fields = get_stats(system)
kernel_params = build_kuramoto_kernel_params(pF, pG)

# Open output writers for statistics and (optionally) full state snapshots
stats_writer = open_stats_writer(
    str(stats_path),
    fieldnames=stats_fields,
)

state_writer = open_state_writer(
    str(state_path),
    dim=len(x0),
)


In [ ]:
# Run the simulation and ensure files are closed even if an error occurs
try:
    x_final, timings = integrate_chunked_jit_timed(
        kuramoto_chunk,
        x0,
        params_int=p_int,
        kernel_params=kernel_params,
        stats_fn=stats_fn,
        stats_writer=stats_writer,
        stats_params=pF,
        state_writer=state_writer,
        # Keep phases in [0, 2pi) to avoid numerical drift
        state_transform=lambda x: np.mod(x, 2 * np.pi),
    )
    
    # Record the configuration used in this run
    params_system_for_config = {k: v for k, v in params_system.items() if k != "A"}
    config_used = {
        "system": {"name": system, "params": params_system_for_config},
        "noise": params_noise,
        "network": {"name": network_name, "params": network_params},
        "integrator": p_int.model_dump(),
    }
    (output_dir / "timings.json").write_text(json.dumps(timings, indent=2)) # write timings for the run
    (output_dir / "config_used.json").write_text(json.dumps(config_used, indent=2)) # write all the parameters choices for the run
finally:
    close_stats_writer(stats_writer)
    close_state_writer(state_writer)


### Multiharmonic all-to-all Kuramoto

In [ ]:
# Standalone integration cell for multiharmonic all-to-all Kuramoto
import json
from pathlib import Path
import numpy as np

from dyn_net.dynamical_systems import get_drift
from dyn_net.dynamical_systems.stats_registry import get_stats
from dyn_net.dynamical_systems.jit_kuramoto_all_to_all import (
    kuramoto_all_to_all_chunk,
    build_kuramoto_all_to_all_kernel_params,
)
from dyn_net.noise import get_noise
from dyn_net.integrator.params import EulerMaruyamaParams
from dyn_net.integrator.jit import integrate_chunked_jit_timed
from dyn_net.networks import get_network
from dyn_net.utils.network import update_system_params_for_network
from dyn_net.utils.stats import open_stats_writer, close_stats_writer
from dyn_net.utils.state import open_state_writer, close_state_writer

# Number of agents
n = 1000

# System parameters
system = "kuramoto_all_to_all"
params_system = {
    "theta": 0.01,
    # coefficients a_n for interaction potential W(x) = -sum a_n cos(n x)
    "a": [1.0],
}

# Noise parameters
params_noise = {
    "sigma": 0.1,
}

# Network parameters (all-to-all uses only n)
network_name = "all-to-all"
network_params = {"n": n}

# Integrator parameters
p_int = EulerMaruyamaParams(
    tmin=0.0,
    tmax=5_000.0,
    dt=0.01,
    stats_every=1000,
    state_every=1000,
    write_stats_at_start=True,
    write_state_at_start=True,
)

# Initial condition (uniform on [0, 2pi))
x0 = np.random.uniform(0.0, 2 * np.pi, size=n)

# Output paths
output_dir = Path('.')
output_dir.mkdir(parents=True, exist_ok=True)
stats_path = output_dir / "stats_kuramoto_all_to_all.h5"
state_path = output_dir / "state_kuramoto_all_to_all.h5"

# Build network and system
build_net, p_net = get_network(network_name, network_params)
A = build_net(p_net)
update_system_params_for_network(params_system, network_name, network_params, A)

F, pF = get_drift(system, params_system)
G, pG = get_noise("additive_gaussian", params_noise)
stats_fn, stats_fields = get_stats(system)
kernel_params = build_kuramoto_all_to_all_kernel_params(pF, pG)

stats_writer = open_stats_writer(str(stats_path), fieldnames=stats_fields)
state_writer = open_state_writer(str(state_path), dim=len(x0))

try:
    params_system_for_config = {k: v for k, v in params_system.items() if k != "A"}
    config_used = {
        "system": {"name": system, "params": params_system_for_config},
        "noise": params_noise,
        "network": {"name": network_name, "params": network_params},
        "integrator": p_int.model_dump(),
    }
    (output_dir / "config_used_kuramoto_all_to_all.json").write_text(json.dumps(config_used, indent=2))
    x_final, timings = integrate_chunked_jit_timed(
        kuramoto_all_to_all_chunk,
        x0,
        params_int=p_int,
        kernel_params=kernel_params,
        stats_fn=stats_fn,
        stats_writer=stats_writer,
        stats_params=pF,
        state_writer=state_writer,
        state_transform=lambda x: np.mod(x, 2 * np.pi),
    )


    (output_dir / "timings_kuramoto_all_to_all.json").write_text(json.dumps(timings, indent=2))
    
finally:
    close_stats_writer(stats_writer)
    close_state_writer(state_writer)
